## LLM Summarisation

LLMs are capable of condensing lengthy documents into concise summaries. They understand and extract the main ideas and key details from a text, presenting them in a clear and coherent manner. This is useful in areas like research, journalism, and daily information management, helping users to quickly grasp essential information from vast amounts of text.

This guide will present several approachs to summarisation with LLMs

## Langchain Summarisation

Langchain allows you to build more formal summarisation chains, using the following common approaches for loading documents into the LLM context window:

- `Stuff` approach: The simplest approach which involves loading all your documents into the context window of the LLM (similar to the above raw direct summarisation).
- `Map-Reduce` approach: This involves a chain with two steps - `map` and `reduce`. Essentially, the chain summarises each document separately during the `map` step and the `reduce`s these summaries into a final summary.

The `Map-Reduce` approach is ideal for scenarios where the documents cannot fit into the LLM context window, so we split them into chunks, summarise those chunks (thereby reducing their size) and then summarise the list of summaries.

<img src='https://python.langchain.com/assets/images/summarization_use_case_2-f2a4d5d60980a79140085fb7f8043217.png'/>

### load_summarize_chain

LangChain provides an optimised chain function called load_summarize_chain. This function abstracts the above two approaches (and uses them underneath). You provide the `chain_type` parameter to switch between `stuff` and `map_reduce`. You can optionally provide your own prompts as well, though it uses an internal prompt for summarisation by default.



In [7]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="mistral", num_ctx=8196)  # Default is 2048)

We can then create a stuff chain using the routines below, and finally invoke it to get our summary of the article:

In [8]:
import re
def cleanup_newlines(docs):
    for doc in docs:
        doc.page_content = re.sub("\n\n+", "\n", doc.page_content)


In [9]:
from langchain_community.document_loaders import WebBaseLoader
docs = WebBaseLoader("https://medium.com/inspiredbrilliance/patterns-for-microservices-e57a2d71ff9e").load()
cleanup_newlines(docs)
llm.get_num_tokens(docs[0].page_content)

3885

In [10]:
from langchain.chains.summarize import load_summarize_chain

stuff_chain = load_summarize_chain(llm, chain_type="stuff")
stuff_chain.invoke(docs)["output_text"]

' The article "Patterns for microservices - Sync vs Async" by Priyank Gupta discusses the choice between synchronous and asynchronous communication styles in designing microservices architectures. The author notes that inter-service communication flow is a crucial decision that impacts various system dimensions, including consumers, workflow management, read/write frequency bias, and more.\n\nFor synchronous communication, components wait for a response before proceeding. It\'s suitable for consumer applications that expect a unified interface and can mask the complexity of a distributed system. However, it lacks flexibility in handling complex workflows and doesn\'t scale well for high read/write frequency systems.\n\nIn contrast, asynchronous communication allows components to decouple their execution by not waiting for a response. It\'s an excellent fit for write-heavy systems and can handle sporadic bursts of traffic. However, it increases system complexity due to the need for mess

Next we begin by loading a larger piece of text, namely Franz Kafka's short story: Metamorphosis.

In [11]:
from langchain_community.document_loaders import WebBaseLoader
metamorphosis_url = "https://www.gutenberg.org/cache/epub/5200/pg5200-images.html"
book_loader = WebBaseLoader(metamorphosis_url)
book_docs = book_loader.load()


In [12]:
print(llm.get_num_tokens(book_docs[0].page_content))

35727


In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=20000,
    chunk_overlap=500
)

split_docs = splitter.split_documents(book_docs)
len(split_docs)

8

In [16]:
map_reduce_chain = load_summarize_chain(llm, chain_type="map_reduce")
map_reduce_chain.invoke(split_docs)["output_text"]

' In "Metamorphosis" by Franz Kafka, the protagonist Gregor Samsa transforms into a giant insect-like creature and is unable to perform his duties as a traveling salesman. His family grows concerned about his absence from work and eventually confronts him, leading to misunderstandings and distress. Gregor tries to protect his family and find a solution but faces physical and emotional challenges. As days pass, the family\'s financial situation worsens, and they struggle to care for Gregor while dealing with their own concerns. Eventually, due to their exhaustion and financial difficulties, they decide to lock Gregor in his room. One day, Gregor is found dead by the cleaner, leaving his family grieving. The story is part of Project Gutenberg, a digital library that distributes free electronic works with contributors providing royalties and donations.'

## Useful Links

https://python.langchain.com/v0.1/docs/use_cases/summarization/